# SMI AL Loop

In [2]:
# print(os.getcwd())
import h5py
import time
import random
import datetime
import copy
import numpy as np
import os
import csv
import json
import subprocess
import sys
import PIL.Image as Image
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from matplotlib import pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
from cords.cords.selectionstrategies.supervisedlearning import DataSelectionStrategy
# from cords.cords.utils.models import ResNet18
from distil.distil.utils.models.resnet import ResNet18
# from ..gable.utils.models import MobileNetV2
from gable.gable.utils.custom_dataset import load_dataset_custom
from torch.utils.data import Subset
from torch.autograd import Variable
import tqdm
from math import floor
from sklearn.metrics import roc_curve, auc, confusion_matrix, precision_recall_curve, roc_auc_score
from sklearn.preprocessing import label_binarize
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
from distil.distil.active_learning_strategies import BADGE, EntropySampling, GLISTER, GradMatchActive, CoreSet, LeastConfidence, MarginSampling 
from distil.distil.utils.data_handler import DataHandler_CIFAR10, DataHandler_MNIST

seed=42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed) 
# for cuda
# torch.cuda.manual_seed_all(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False
# torch.backends.cudnn.enabled = False

In [3]:
torch.cuda.empty_cache()

In [4]:
from torch.utils.data import Dataset
class custom_subset(Dataset):
    r"""
    Subset of a dataset at specified indices.

    Arguments:
        dataset (Dataset): The whole Dataset
        indices (sequence): Indices in the whole set selected for subset
        labels(sequence) : targets as required for the indices. will be the same length as indices
    """
    def __init__(self, dataset, indices, labels):
        self.dataset = torch.utils.data.Subset(dataset, indices)
        self.targets = labels.type(torch.long)
    def __getitem__(self, idx):
        image = self.dataset[idx][0]
        target = self.targets[idx]
        return (image, target)

    def __len__(self):
        return len(self.targets)

In [5]:
def model_eval_loss(data_loader, model, criterion):
    total_loss = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(data_loader):
            inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss

def init_weights(m):
#     torch.manual_seed(35)
    if isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform_(m.weight)
    elif isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

def weight_reset(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()
                
def create_model(name, num_cls, device):
    if name == 'ResNet18':
        model = ResNet18(num_cls)
    elif name == 'MnistNet':
        model = MnistNet()
    elif name == 'ResNet164':
        model = ResNet164(num_cls)
    elif name == 'MobileNetV2':
        model = MobileNetV2(num_cls)
#         model = models.mobilenet_v2(pretrained=True)
#         model.classifier = nn.Sequential(
#             nn.Dropout(0.2),
#             nn.Linear(model.last_channel, num_cls),
#         ) 
    model.apply(init_weights)
    model = model.to(device)
    return model

def loss_function():
    criterion = nn.CrossEntropyLoss()
    criterion_nored = nn.CrossEntropyLoss(reduction='none')
    return criterion, criterion_nored

def optimizer_with_scheduler(model, num_epochs, learning_rate, m=0.9, wd=5e-4):
    optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                          momentum=m, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    return optimizer, scheduler

def optimizer_without_scheduler(model, learning_rate, m=0.9, wd=5e-4):
#     optimizer = optim.Adam(model.parameters(),weight_decay=wd)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                          momentum=m, weight_decay=wd)
    return optimizer

def generate_cumulative_timing(mod_timing):
    tmp = 0
    mod_cum_timing = np.zeros(len(mod_timing))
    for i in range(len(mod_timing)):
        tmp += mod_timing[i]
        mod_cum_timing[i] = tmp
    return mod_cum_timing/3600

def kernel(x, y, measure="cosine", exp=2):
    if(measure=="eu_sim"):
        dist = pairwise_distances(x.cpu().numpy(), y.cpu().numpy())
        sim = max(dist.ravel()) - dist
#         n = x.size(0)
#         m = y.size(0)
#         d = x.size(1)
#         x = x.unsqueeze(1).expand(n, m, d)
#         y = y.unsqueeze(0).expand(n, m, d)
#         dist = torch.pow(x - y, exp).sum(2)
#         const = torch.max(dist).item()
#         sim = (const - dist)
    
        #dist = torch.exp(-1 * torch.pow(x - y, 2).sum(2))
    if(measure=="cosine"):
        sim = cosine_similarity(x.cpu().numpy(), y.cpu().numpy())
    return sim


def save_kernel_hdf5(lake_kernel, lake_target_kernel, target_kernel=[], lake_private_kernel=[], pp_kernel=[], numpy=True):
    if(not(numpy)):
        lake_kernel = lake_kernel.cpu().numpy()
    with h5py.File("smi_lake_kernel_" + device.split(":")[1] +".hdf5", 'w') as hf:
        hf.create_dataset("kernel",  data=lake_kernel)
    if(not(numpy)):
        lake_target_kernel = lake_target_kernel.cpu().numpy()
    with h5py.File("smi_lake_target_kernel_" + device.split(":")[1] + ".hdf5", 'w') as hf:
        hf.create_dataset("kernel",  data=lake_target_kernel)
    if(not(numpy)):
        target_kernel = target_kernel.cpu().numpy()
    with h5py.File("smi_target_kernel_" + device.split(":")[1] + ".hdf5", 'w') as hf:
        hf.create_dataset("kernel",  data=target_kernel)
    if(not(numpy)):
        lake_private_kernel = lake_private_kernel.cpu().numpy()
    with h5py.File("smi_lake_private_kernel_" + device.split(":")[1] + ".hdf5", 'w') as hf:
        hf.create_dataset("kernel",  data=lake_private_kernel)
    if(not(numpy)):
        pp_kernel = pp_kernel.cpu().numpy()
    with h5py.File("smi_pp_kernel_" + device.split(":")[1] + ".hdf5", 'w') as hf:
        hf.create_dataset("kernel",  data=pp_kernel)
            
def find_err_per_class(test_set, val_set, final_val_classifications, final_val_predictions, final_tst_classifications, 
                       final_tst_predictions, saveDir, prefix):
    #find queries from the validation set that are erroneous
#     saveDir = os.path.join(saveDir, prefix)
#     if(not(os.path.exists(saveDir))):
#         os.mkdir(saveDir)
    val_err_idx = list(np.where(np.array(final_val_classifications) == False)[0])
    tst_err_idx = list(np.where(np.array(final_tst_classifications) == False)[0])
    val_class_err_idxs = []
    tst_err_log = []
    val_err_log = []
    for i in range(num_cls):
        tst_class_idxs = list(torch.where(torch.Tensor(test_set.targets) == i)[0].cpu().numpy())
        val_class_idxs = list(torch.where(torch.Tensor(val_set.targets.float()) == i)[0].cpu().numpy())
        #err classifications per class
        val_err_class_idx = set(val_err_idx).intersection(set(val_class_idxs))
        tst_err_class_idx = set(tst_err_idx).intersection(set(tst_class_idxs))
        if(len(val_class_idxs)>0):
            val_error_perc = round((len(val_err_class_idx)/len(val_class_idxs))*100,2)
        else:
            val_error_perc = 0
        tst_error_perc = round((len(tst_err_class_idx)/len(tst_class_idxs))*100,2)
        print("val, test error% for class ", i, " : ", val_error_perc, tst_error_perc)
        val_class_err_idxs.append(val_err_class_idx)
        tst_err_log.append(tst_error_perc)
        val_err_log.append(val_error_perc)
    tst_err_log.append(sum(tst_err_log)/len(tst_err_log))
    val_err_log.append(sum(val_err_log)/len(val_err_log))
    return tst_err_log, val_err_log, val_class_err_idxs


def aug_train_subset(train_set, lake_set, true_lake_set, subset, lake_subset_idxs, budget, augrandom=False):
    all_lake_idx = list(range(len(lake_set)))
    if(not(len(subset)==budget) and augrandom):
        print("Budget not filled, adding ", str(int(budget) - len(subset)), " randomly.")
        remain_budget = int(budget) - len(subset)
        remain_lake_idx = list(set(all_lake_idx) - set(subset))
        random_subset_idx = list(np.random.choice(np.array(remain_lake_idx), size=int(remain_budget), replace=False))
        subset += random_subset_idx
    lake_ss = custom_subset(true_lake_set, subset, torch.Tensor(true_lake_set.targets.float())[subset])
    if(feature=="ood"): 
        ood_lake_idx = list(set(lake_subset_idxs)-set(subset))
        private_set =  custom_subset(true_lake_set, ood_lake_idx, torch.Tensor(np.array([split_cfg['num_cls_idc']]*len(ood_lake_idx))).float())
    remain_lake_idx = list(set(all_lake_idx) - set(lake_subset_idxs))
    remain_lake_set = custom_subset(lake_set, remain_lake_idx, torch.Tensor(lake_set.targets.float())[remain_lake_idx])
    remain_true_lake_set = custom_subset(true_lake_set, remain_lake_idx, torch.Tensor(true_lake_set.targets.float())[remain_lake_idx])
    print(len(lake_ss),len(remain_lake_set),len(lake_set))
    if(feature!="ood"): assert((len(lake_ss)+len(remain_lake_set))==len(lake_set))
    aug_train_set = torch.utils.data.ConcatDataset([train_set, lake_ss])
    if(feature=="ood"): 
        return aug_train_set, remain_lake_set, remain_true_lake_set, private_set, lake_ss
    else:
        return aug_train_set, remain_lake_set, remain_true_lake_set
                        
def getMisclsSet(val_set, val_class_err_idxs, imb_cls_idx):
    miscls_idx = []
    for i in range(len(val_class_err_idxs)):
        if i in imb_cls_idx:
            miscls_idx += val_class_err_idxs[i]
    print("total misclassified ex from imb classes: ", len(miscls_idx))
    return Subset(val_set, miscls_idx)

def getMisclsSetNumpy(X_val, y_val, val_class_err_idxs, imb_cls_idx):
    miscls_idx = []
    for i in range(len(val_class_err_idxs)):
        if i in imb_cls_idx:
            miscls_idx += val_class_err_idxs[i]
    print("total misclassified ex from imb classes: ", len(miscls_idx))
    return X_val[miscls_idx], y_val[miscls_idx]

def getPrivateSet(lake_set, subset, private_set):
    #augment prev private set and current subset
    new_private_set = custom_subset(lake_set, subset, torch.Tensor(lake_set.targets.float())[subset])
#     new_private_set =  Subset(lake_set, subset)
    total_private_set = torch.utils.data.ConcatDataset([private_set, new_private_set])
    return total_private_set

def getSMI_ss(datkbuildPath, exePath, hdf5Path, budget, numQueries, sf):
    if(sf=="fl1mi"):
        command = os.path.join(datkbuildPath, exePath) + " -mode query -naiveOrRandom naive -magnificationLambda 1 -numSummaries 1 -budget " + str(budget) + " -queryPrivacyOptimizer " + sf + " -numQueries " + numQueries + " -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path, "smi_lake_kernel_"+str(device).split(":")[1]+".hdf5") +  " -queryKernelFile " + os.path.join(hdf5Path, "smi_lake_target_kernel_"+str(device).split(":")[1]+".hdf5")
    elif(sf == "logdetmi"):
        command = os.path.join(datkbuildPath, "cifarSubsetSelector_ng") + " -mode query -naiveOrRandom naive -logDetLambda 1 -magnificationLambda 1 -numSummaries 1 -budget " + str(budget) + " -queryPrivacyOptimizer " + sf + " -numQueries  " + numQueries + "  -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path, "smi_lake_kernel_"+str(device).split(":")[1]+".hdf5") + " -queryKernelFile " + os.path.join(hdf5Path, "smi_lake_target_kernel_"+str(device).split(":")[1]+".hdf5") + " -queryqueryKernelFile " + os.path.join(hdf5Path, "smi_target_kernel_"+str(device).split(":")[1]+".hdf5")
    elif(sf=="fl2mi"):
        command = os.path.join(datkbuildPath, exePath) + " -mode query -naiveOrRandom naive -queryDiversityLambda 1 -magnificationLambda 1 -numSummaries 1 -budget " + str(budget) + " -queryPrivacyOptimizer " + sf + " -numQueries  " + numQueries + " -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path, "smi_lake_kernel_"+str(device).split(":")[1]+".hdf5") + " -queryKernelFile " + os.path.join(hdf5Path, "smi_lake_target_kernel_"+str(device).split(":")[1]+".hdf5")
    elif(sf=="gcmi" or sf=="div-gcmi"):
        command = os.path.join(datkbuildPath, exePath) + " -mode query -naiveOrRandom naive -magnificationLambda 1 -numSummaries 1 -budget " + str(budget) + " -queryPrivacyOptimizer " + sf + " -numQueries " + numQueries + " -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path,"smi_lake_kernel_"+str(device).split(":")[1]+".hdf5") + " -queryKernelFile " + os.path.join(hdf5Path,"smi_lake_target_kernel_"+str(device).split(":")[1]+".hdf5")
    elif(sf=="gccg"):
        command = os.path.join(datkbuildPath, exePath) + " -mode private -naiveOrRandom naive -gcLambda 1 -magnificationLambda 1 -numSummaries 1 -budget " + str(budget) + " -queryPrivacyOptimizer " + sf + " -numPrivates " + numQueries + " -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path,"smi_lake_kernel_"+str(device).split(":")[1]+".hdf5") + " -privateKernelFile " + os.path.join(hdf5Path,"smi_lake_target_kernel_"+str(device).split(":")[1]+".hdf5")
    elif(sf=="fl1cg"):
        command = os.path.join(datkbuildPath, exePath) + " -mode private -naiveOrRandom naive -magnificationLambda 1 -numSummaries 1 -budget " + str(budget) + " -queryPrivacyOptimizer " + sf + " -numPrivates " + numQueries + " -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path,"smi_lake_kernel_"+str(device).split(":")[1]+".hdf5") + " -privateKernelFile " + os.path.join(hdf5Path,"smi_lake_target_kernel_"+str(device).split(":")[1]+".hdf5")
    elif(sf=="logdetcg"):
        command = os.path.join(datkbuildPath, exePath) + " -mode private -naiveOrRandom naive -magnificationLambda 1 -numSummaries 1 -budget " + str(budget) + " -queryPrivacyOptimizer " + sf + " -numPrivates " + numQueries + " -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path,"smi_lake_kernel_"+str(device).split(":")[1]+".hdf5") + " -privateKernelFile " + os.path.join(hdf5Path,"smi_lake_target_kernel_"+str(device).split(":")[1]+".hdf5") + " -privateprivateKernelFile " + os.path.join(hdf5Path, "smi_target_kernel_"+str(device).split(":")[1]+".hdf5")
    elif(sf=="fl" or sf=="logdet"):
        command = os.path.join(datkbuildPath, "cifarSubsetSelector_ng") + " -mode generic -naiveOrRandom naive -logDetLambda 1 -numSummaries 1 -budget " + str(budget) + " -genericOptimizer " + sf + " -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path,"smi_lake_kernel_"+str(device).split(":")[1]+".hdf5")
    elif(sf =="gc"):
        command = os.path.join(datkbuildPath, exePath) + " -mode generic -naiveOrRandom naive -gcLambda 1 -numSummaries 1 -budget " + str(budget) + " -genericOptimizer " + sf + " -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path,"smi_lake_kernel_"+str(device).split(":")[1]+".hdf5")
    elif(sf =="dsum"):
        command = os.path.join(datkbuildPath, exePath) + " -mode generic -naiveOrRandom naive -gcLambda 1 -numSummaries 1 -budget " + str(budget) + " -genericOptimizer " + sf + " -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path,"smi_lake_kernel_"+str(device).split(":")[1]+".hdf5")
    print("Executing SIM command: ", command)
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=True, shell=True)
    subset = process.communicate()[0]
    subset = subset.decode("utf-8")
    subset = subset.strip().split(" ")
    subset = list(map(int, subset))
    return subset

def getCMI_ss(datkbuildPath, exePath, hdf5Path, budget, numQueries, numPrivates, sf):
    if(sf=="flmic"):
        command = os.path.join(datkbuildPath, exePath) + " -mode joint -naiveOrRandom naive -magnificationLambda 1 -numSummaries 1 -budget " + str(budget) + " -queryPrivacyOptimizer " + sf + " -numQueries " + numQueries + " -numPrivates " + numPrivates + " -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path, "smi_lake_kernel_"+str(device).split(":")[1]+".hdf5") +  " -queryKernelFile " + os.path.join(hdf5Path, "smi_lake_target_kernel_"+str(device).split(":")[1]+".hdf5") + " -privateKernelFile " + os.path.join(hdf5Path,"smi_lake_private_kernel_"+str(device).split(":")[1]+".hdf5")
    if(sf == "logdetmic"):
        command = os.path.join(datkbuildPath, exePath) + " -mode joint -naiveOrRandom naive -magnificationLambda 1 -numSummaries 1 -budget " + str(budget) + " -queryPrivacyOptimizer " + sf + " -numQueries " + numQueries + " -numPrivates " + numPrivates + " -dontComputeKernel true -imageKernelFile " + os.path.join(hdf5Path, "smi_lake_kernel_"+str(device).split(":")[1]+".hdf5") +  " -queryKernelFile " + os.path.join(hdf5Path, "smi_lake_target_kernel_"+str(device).split(":")[1]+".hdf5") + " -privateKernelFile " + os.path.join(hdf5Path,"smi_lake_private_kernel_"+str(device).split(":")[1]+".hdf5")
    
    print("Executing CMI command: ", command)
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=True, shell=True)
    subset = process.communicate()[0]
    subset = subset.decode("utf-8")
    subset = subset.strip().split(" ")
    subset = list(map(int, subset))
    return subset

def remove_ood_points(lake_set, subset, idc_idx):
    idx_subset = []
    subset_cls = torch.Tensor(lake_set.targets.float())[subset]
    for i in idc_idx:
        idc_subset_idx = list(torch.where(subset_cls == i)[0].cpu().numpy())
        idx_subset += list(np.array(subset)[idc_subset_idx])
    print(len(idx_subset),"/",len(subset), " idc points.")
    return idx_subset

def getPerClassSel(lake_set, subset, num_cls):
    perClsSel = []
    subset_cls = torch.Tensor(lake_set.targets.float())[subset]
    for i in range(num_cls):
        cls_subset_idx = list(torch.where(subset_cls == i)[0].cpu().numpy())
        perClsSel.append(len(cls_subset_idx))
    return perClsSel

#check overlap with prev selections
def check_overlap(prev_idx, prev_idx_hist, idx):
    prev_idx = [int(x/num_rep) for x in prev_idx]
    prev_idx_hist = [int(x/num_rep) for x in prev_idx_hist]
    idx = [int(x/num_rep) for x in idx]
    # overlap = set(prev_idx).intersection(set(idx))
    overlap = [value for value in idx if value in prev_idx] 
    # overlap_hist = set(prev_idx_hist).intersection(set(idx))
    overlap_hist = [value for value in idx if value in prev_idx_hist]
    new_points = set(idx) - set(prev_idx_hist)
    total_unique_points = set(idx+prev_idx_hist)
    print("New unique points: ", len(new_points))
    print("Total unique points: ", len(total_unique_points))
    print("overlap % of sel with prev idx: ", len(overlap)/len(idx))
    print("overlap % of sel with all prev idx: ", len(overlap_hist)/len(idx))
    return len(overlap)/len(idx), len(overlap_hist)/len(idx)

def getFeatures(model, dataloader):
    pass

def get_roc_auc(target, output, n_classes):
    target = label_binarize(target, classes=list(range(n_classes)))
    output = np.array(output)
#     print('Targets: ',target,' Outputs: ',output)
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(target[:,i],output[:,i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        fpr[i] = fpr[i].tolist()
        tpr[i] = tpr[i].tolist()
        roc_auc[i] = roc_auc[i].tolist()
#     fig, ax = plt.subplots(figsize=(4,4))
#     ax.plot([0, 1], [0, 1], 'k--')
#     ax.set_xlim([0.0, 1.0])
#     ax.set_ylim([0.0, 1.05])
#     ax.set_xlabel('False Positive Rate')
#     ax.set_ylabel('True Positive Rate')
#     ax.set_title('Receiver operating characteristic example')
#     for i in range(n_classes):
#         ax.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f) for label %i' % (roc_auc[i], i))
#     ax.legend(loc="best")
#     ax.grid(alpha=.4)
#     plt.show()
    return fpr,tpr,roc_auc
    
def get_pr_auc(target, output, n_classes):
    target = label_binarize(target, classes=list(range(n_classes)))
    output = np.array(output)
#     print('Targets: ',target,' Outputs: ',output)
    precision = dict()
    recall = dict()
    aupr = dict()
    for i in range(n_classes):
        precision[i], recall[i], _ = precision_recall_curve(target[:,i],output[:,i])
        aupr[i] = auc(recall[i], precision[i])
        precision[i] = precision[i].tolist()
        recall[i] = recall[i].tolist()
        aupr[i] = aupr[i].tolist()
#     fig, ax = plt.subplots(figsize=(4,4))
#     ax.plot([0, 1], [0, 1], 'k--')
#     ax.set_xlim([0.0, 1.0])
#     ax.set_ylim([0.0, 1.05])
#     ax.set_xlabel('Recall')
#     ax.set_ylabel('Precision')
#     ax.set_title('Receiver operating characteristic example')
#     for i in range(n_classes):
#         ax.plot(recall[i], precision[i], label='Precision Recall curve (area = %0.2f) for label %i' % (aupr[i], i))
#     ax.legend(loc="best")
#     ax.grid(alpha=.4)
#     plt.show()
    return precision, recall, aupr

def get_macro_roc_auc(target, output):
    target = label_binarize(target,classes=list(range(num_cls)))
    output = np.array(output)
    return roc_auc_score(target,output,average="macro")
    

In [15]:
datadir = '../data'
data_name = 'cifar10'
num_cls=10
fraction = float(0.1)
budget=250
num_epochs = int(10)
num_rep = 10
model_name = 'ResNet18'
learning_rate = 0.01
feature = 'longtail'
split_cfg = {"num_cls_imbalance":10,
             "sel_cls_idx":[0,1,2,3,4,5,6,7,8,9],
             "per_imbclass_train":{0:3,1:6,2:12,3:24,4:48,5:96,6:192,7:384,8:768,9:1536}, 
             "per_imbclass_val":{0:10,1:10,2:10,3:10,4:10,5:10,6:10,7:10,8:10,9:10},
             "per_imbclass_lake":{0:60,1:120,2:240,3:480,4:960,5:960,6:1920,7:3840,8:4222,9:3454}, #0-4:20x, 5-7:10x, 8-9:Remaining 
            } #cifar10
initModelPath = "weights/"+data_name + "_" + model_name + "_" + str(learning_rate) + "_" + str(split_cfg["per_imbclass_train"])
num_runs = 1  # number of random runs
computeClassErrorLog = True
run=1
magnification = 1
device = "cuda:0" if torch.cuda.is_available() else "cpu"
datkbuildPath = "/scratch4/snk170001/code/datk/build"
exePath = "cifarSubsetSelector"
print("Using Device:", device)
doublePrecision = True
linearLayer = True
handler = DataHandler_CIFAR10


Using Device: cuda:0


# AL Like Train Loop

In [16]:
def train_model_al(datkbuildPath, exePath, num_epochs, dataset_name, datadir, feature, model_name, budget, split_cfg, learning_rate, run,
                device, computeErrorLog, strategy="SIM", sf=""):
#     torch.manual_seed(42)
#     np.random.seed(42)
    print(strategy, sf)
    #load the dataset based on type of feature
    if(feature=="classimb" or feature=="longtail" or feature=="ood"):
        if(strategy == "SIM" or strategy == "SF" or strategy=="random"):
            if(strategy == "SF" or strategy=="random"):
                train_set, val_set, test_set, lake_set, sel_cls_idx, num_cls = load_dataset_custom(datadir, dataset_name, feature, split_cfg, False, True)
            else:
                train_set, val_set, test_set, lake_set, sel_cls_idx, num_cls = load_dataset_custom(datadir, dataset_name, feature, split_cfg, False, False)
        elif(strategy=="AL"):
            if(sf=="badge" or sf=="us"):
                X_tr, y_tr, X_val, y_val, X_unlabeled, y_unlabeled, train_set, val_set, test_set, lake_set, sel_cls_idx, num_cls = load_dataset_custom(datadir, dataset_name, feature, split_cfg, True, False)
            else: #dont augment train with valid
                X_tr, y_tr, X_val, y_val, X_unlabeled, y_unlabeled, train_set, val_set, test_set, lake_set, sel_cls_idx, num_cls = load_dataset_custom(datadir, dataset_name, feature, split_cfg, True, False)
        print("selected classes are: ", sel_cls_idx)
    if(feature=="duplicate" or feature=="vanilla"):
        sel_cls_idx = None
        if(strategy == "SIM" or strategy=="random"):
            train_set, val_set, test_set, lake_set, num_cls = load_dataset_custom(datadir, dataset_name, feature, split_cfg)
        elif(strategy=="AL"):
            X_tr, y_tr, X_val, y_val, X_unlabeled, y_unlabeled, train_set, val_set, test_set, lake_set, num_cls = load_dataset_custom(datadir, dataset_name, feature, split_cfg, True)
    if(feature=="ood"): num_cls+=1 #Add one class for OOD class
    N = len(train_set)
    trn_batch_size = 10
    val_batch_size = 10
    tst_batch_size = 10

    trainloader = torch.utils.data.DataLoader(train_set, batch_size=trn_batch_size,
                                              shuffle=True, pin_memory=True)

    valloader = torch.utils.data.DataLoader(val_set, batch_size=val_batch_size, 
                                            shuffle=False, pin_memory=True)

    tstloader = torch.utils.data.DataLoader(test_set, batch_size=tst_batch_size,
                                             shuffle=False, pin_memory=True)
    
    lakeloader = torch.utils.data.DataLoader(lake_set, batch_size=tst_batch_size,
                                         shuffle=False, pin_memory=True)
    true_lake_set = copy.deepcopy(lake_set)
    # Budget for subset selection
    bud = budget
   
    # Variables to store accuracies
    fulltrn_losses = np.zeros(num_epochs)
    val_losses = np.zeros(num_epochs)
    tst_losses = np.zeros(num_epochs)
    timing = np.zeros(num_epochs)
    val_acc = np.zeros(num_epochs)
    full_trn_acc = np.zeros(num_epochs)
    tst_acc = np.zeros(num_epochs)
    final_tst_predictions = []
    final_tst_classifications = []
    best_val_acc = -1
    csvlog = []
    val_csvlog = []
    # Results logging file
    print_every = 3
    all_logs_dir = 'active_learning_results/' + dataset_name  + '/' + feature + '/'+  sf + '/' + str(bud) + '/' + str(run)
    print("Saving results to: ", all_logs_dir)
    subprocess.run(["mkdir", "-p", all_logs_dir])
    exp_name = dataset_name + "_" + feature +  "_" + strategy + "_" + str(len(sel_cls_idx))  +"_" + sf +  '_budget:' + str(bud) + '_epochs:' + str(num_epochs) + '_linear:'  + str(linearLayer) + '_runs' + str(run)
    print(exp_name)
    res_dict = {"dataset":data_name, "feature":feature, "sel_func":sf, "sel_budget":budget, "num_selections":num_epochs, "model":model_name, "learning_rate":learning_rate, "setting":split_cfg, "all_class_acc":None, "test_acc":[],"sel_per_cls":[], "sel_cls_idx":sel_cls_idx, "trn_fpr":[], "trn_tpr":[], "trn_roc_auc":[], "val_fpr":[], "val_tpr":[], "val_roc_auc":[], "tst_fpr":[], "tst_tpr":[], "tst_roc_auc":[], "trn_precision":[], "trn_recall":[], "trn_aupr":[], "val_precision":[], "val_recall":[], "val_aupr":[], "tst_precision":[], "tst_recall":[], "tst_aupr":[], "macro_avg_roc":[]}
    # Model Creation
    model = create_model(model_name, num_cls, device)
    model1 = create_model(model_name, num_cls, device)
    if(strategy == "AL"):
        strategy_args = {'batch_size' : budget, 'lr':float(0.001), 'device':device}
        if(sf=="badge"):
            strategy_sel = BADGE(X_tr, y_tr, X_unlabeled, model, handler, num_cls, strategy_args)
        elif(sf=="us"):
            strategy_sel = EntropySampling(X_tr, y_tr, X_unlabeled, model, handler, num_cls, strategy_args)
        elif(sf=="glister" or sf=="glister-tss"):
            strategy_sel = GLISTER(X_tr, y_tr, X_unlabeled, model, handler, num_cls, strategy_args, valid=True, X_val=X_val, Y_val=y_val, typeOf='rand', lam=0.1)
        elif(sf=="gradmatch-tss"):
            strategy_args = {'batch_size' : 1, 'lr':float(0.01)}
            strategy_sel = GradMatchActive(X_tr, y_tr, X_unlabeled, model, F.cross_entropy, handler, num_cls, strategy_args["lr"], "PerBatch", False, strategy_args, valid=True, X_val=X_val, Y_val=y_val)
        elif(sf=="coreset"):
            strategy_sel = CoreSet(X_tr, y_tr, X_unlabeled, model, handler, num_cls, strategy_args)
        elif(sf=="leastconf"):
            strategy_sel = LeastConfidence(X_tr, y_tr, X_unlabeled, model, handler, num_cls, strategy_args)
        elif(sf=="margin"):
            strategy_sel = MarginSampling(X_tr, y_tr, X_unlabeled, model, handler, num_cls, strategy_args)
    # Loss Functions
    criterion, criterion_nored = loss_function()

    # Getting the optimizer and scheduler
    optimizer, scheduler = optimizer_with_scheduler(model, 150, learning_rate)
#     optimizer = optimizer_without_scheduler(model, learning_rate)
    private_set = []
    #overlap vars
    prev_idx = None
    prev_idx_hist = []
    sel_hist = []
    per_ep_overlap = []
    overall_overlap = []
    idx_tracker = np.array(list(range(len(lake_set))))
    #kernels
    train_private_kernel = []
    train_val_kernel = []
    pp_kernel = []
    val_kernel = []
    
    for i in range(num_epochs):
        print("AL epoch: ", i)
        tst_loss = 0
        tst_correct = 0
        tst_total = 0
        val_loss = 0
        val_correct = 0
        val_total = 0
        
        if(i==0):
            print("initial training epoch")
            if(os.path.exists(initModelPath)):
                model.load_state_dict(torch.load(initModelPath, map_location=device))
                print("Init model loaded from disk, skipping init training: ", initModelPath)
                with torch.no_grad():
                    final_val_predictions = []
                    final_val_classifications = []
                    final_val_targets = []
                    final_val_outputs = []
                    for batch_idx, (inputs, targets) in enumerate(valloader):
                        inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
                        outputs = model(inputs)
                        loss = criterion(outputs, targets)
                        val_loss += loss.item()
                        _, predicted = outputs.max(1)
                        val_total += targets.size(0)
                        val_correct += predicted.eq(targets).sum().item()
                        final_val_predictions += list(predicted.cpu().numpy())
                        final_val_classifications += list(predicted.eq(targets).cpu().numpy())
                        final_val_targets += list(targets.cpu().numpy())
                        final_val_outputs += list(outputs.cpu().numpy().tolist())
                    val_fpr, val_tpr, val_roc_auc = get_roc_auc(final_val_targets,final_val_outputs,num_cls)
                    val_precision, val_recall, val_aupr = get_pr_auc(final_val_targets,final_val_outputs,num_cls)
                    res_dict["val_fpr"].append(val_fpr)
                    res_dict["val_tpr"].append(val_tpr)
                    res_dict["val_roc_auc"].append(val_roc_auc)
                    res_dict["val_precision"].append(val_precision)
                    res_dict["val_recall"].append(val_recall)
                    res_dict["val_aupr"].append(val_aupr)
  
                    if((val_correct/val_total) > best_val_acc):
                        final_tst_predictions = []
                        final_tst_classifications = []
                    final_tst_targets = []
                    final_tst_outputs = []
                    for batch_idx, (inputs, targets) in enumerate(tstloader):
                        inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
                        outputs = model(inputs)
                        loss = criterion(outputs, targets)
                        tst_loss += loss.item()
                        _, predicted = outputs.max(1)
                        tst_total += targets.size(0)
                        tst_correct += predicted.eq(targets).sum().item()
                        final_tst_targets += list(targets.cpu().numpy())
                        final_tst_outputs += list(outputs.cpu().numpy().tolist())
                        if((val_correct/val_total) > best_val_acc):
                            final_tst_predictions += list(predicted.cpu().numpy())
                            final_tst_classifications += list(predicted.eq(targets).cpu().numpy())  
                    tst_fpr, tst_tpr, tst_roc_auc = get_roc_auc(final_tst_targets,final_tst_outputs,num_cls)
                    tst_precision, tst_recall, tst_aupr = get_pr_auc(final_tst_targets,final_tst_outputs,num_cls)
                    macro_avg_roc = get_macro_roc_auc(final_tst_targets,final_tst_outputs)
                    res_dict["tst_fpr"].append(tst_fpr)
                    res_dict["tst_tpr"].append(tst_tpr)
                    res_dict["tst_roc_auc"].append(tst_roc_auc)
                    res_dict["tst_precision"].append(tst_precision)
                    res_dict["tst_recall"].append(tst_recall)
                    res_dict["tst_aupr"].append(tst_aupr)
                    res_dict["macro_avg_roc"].append(macro_avg_roc)
                    if((val_correct/val_total) > best_val_acc):
                        best_val_acc = (val_correct/val_total)
                    val_acc[i] = val_correct / val_total
                    tst_acc[i] = tst_correct / tst_total
                    val_losses[i] = val_loss
                    tst_losses[i] = tst_loss
                    res_dict["test_acc"].append(tst_acc[i])
                continue
        else:
#             if(full_trn_acc[i-1] >= 0.99): #The model has already trained on the seed dataset
            #use misclassifications on validation set as queries
            #compute hypothesized labels
            hyp_lake_labels = []
            for batch_idx, (inputs, _) in enumerate(lakeloader):
                inputs = inputs.to(device)
                outputs = model(inputs)
                _, predicted = outputs.max(1)
                hyp_lake_labels += list(predicted)
            lake_set = custom_subset(lake_set, list(range(len(hyp_lake_labels))), torch.Tensor(hyp_lake_labels))
            lakeloader = torch.utils.data.DataLoader(lake_set, batch_size=tst_batch_size, shuffle=False, pin_memory=True)
#             sys.exit()
            #compute the error log before every selection
            if(computeErrorLog):
                tst_err_log, val_err_log, val_class_err_idxs = find_err_per_class(test_set, val_set, final_val_classifications, final_val_predictions, final_tst_classifications, final_tst_predictions, all_logs_dir, sf+"_"+str(bud))
                csvlog.append(tst_err_log)
                val_csvlog.append(val_err_log)
            ####SIM####
            if(strategy=="SIM" or strategy=="SF"):
                if(sf.endswith("mi")):
                    if(feature=="classimb" or feature=="longtail"):
                        #make a dataloader for the misclassifications - only for experiments with targets
                        miscls_set = getMisclsSet(val_set, val_class_err_idxs, list(range(num_cls)))
                        misclsloader = torch.utils.data.DataLoader(miscls_set, batch_size=1, shuffle=False, pin_memory=True)
                        setf_model = DataSelectionStrategy(lakeloader, misclsloader, model1, num_cls, linearLayer, F.cross_entropy, device) #set last arg to true for linear layer
                    else:
                        setf_model = DataSelectionStrategy(lakeloader, valloader, model1, num_cls, linearLayer, F.cross_entropy, device)
                elif(sf.endswith("cg")): #atleast one selection must be done for private set in cond gain functions
                    if(len(private_set)!=0):
                        privateSetloader = torch.utils.data.DataLoader(private_set, batch_size=1, shuffle=False, pin_memory=True)
                        setf_model = DataSelectionStrategy(lakeloader, privateSetloader, model1, num_cls, linearLayer, F.cross_entropy, device) #set last arg to true for linear layer
                    else:
                        #compute subset with private set a NULL
                        setf_model = DataSelectionStrategy(lakeloader, valloader, model1, num_cls, linearLayer, F.cross_entropy, device)
                elif(sf.endswith("mic")): #configured for the OOD setting
                    setf_model = DataSelectionStrategy(lakeloader, valloader, model1, num_cls, linearLayer, F.cross_entropy, device) #In-dist samples are in Val
                    if(len(private_set)!=0):
                        privateSetloader = torch.utils.data.DataLoader(private_set, batch_size=1, shuffle=False, pin_memory=True)
                        setf_model_private = DataSelectionStrategy(privateSetloader, privateSetloader, model1, num_cls, linearLayer, F.cross_entropy, device)
                else:
                    setf_model = DataSelectionStrategy(lakeloader, valloader, model1, num_cls, linearLayer, F.cross_entropy, device)
                start_time = time.time()
                cached_state_dict = copy.deepcopy(model.state_dict())
                clone_dict = copy.deepcopy(model.state_dict())
                #update the selection strategy model with new params for gradient computation
                setf_model.update_model(clone_dict)
                if(sf.endswith("mic") and len(private_set)!=0): setf_model_private.update_model(clone_dict)
                if(sf.endswith("mi")): #SMI functions need the target set gradients
                    setf_model.compute_gradients(valid=True, batch=False, perClass=False)
                    print("train minibatch gradients shape ", setf_model.grads_per_elem.shape)
#                     print(setf_model.grads_per_elem)
                    print("val minibatch gradients shape ", setf_model.val_grads_per_elem.shape)
#                     print(setf_model.val_grads_per_elem)
                    if(doublePrecision):
                        train_val_kernel = kernel(setf_model.grads_per_elem.double(), setf_model.val_grads_per_elem.double())#img_query_kernel
                    else:
                        train_val_kernel = kernel(setf_model.grads_per_elem, setf_model.val_grads_per_elem)#img_query_kernel
                    numQueryPrivate = train_val_kernel.shape[1]
                elif(sf.endswith("cg")):
                    if(len(private_set)!=0):
                        setf_model.compute_gradients(valid=True, batch=False, perClass=False)
                        print("train minibatch gradients shape ", setf_model.grads_per_elem.shape)
                        print("val minibatch gradients shape ", setf_model.val_grads_per_elem.shape)
                        if(doublePrecision):
                            train_val_kernel = kernel(setf_model.grads_per_elem.double(), setf_model.val_grads_per_elem.double())#img_query_kernel
                        else:
                            train_val_kernel = kernel(setf_model.grads_per_elem, setf_model.val_grads_per_elem)#img_query_kernel
                        numQueryPrivate = train_val_kernel.shape[1]
                    else:
#                         assert(((i + 1)/select_every)==1)
                        setf_model.compute_gradients(valid=False, batch=False, perClass=False)
                        train_val_kernel = []
                        numQueryPrivate = 0
                elif(sf.endswith("mic")):
                    setf_model.compute_gradients(valid=True, batch=False, perClass=False)
                    print("train minibatch gradients shape ", setf_model.grads_per_elem.shape)
                    print("val minibatch gradients shape ", setf_model.val_grads_per_elem.shape)
                    if(doublePrecision):
                        train_val_kernel = kernel(setf_model.grads_per_elem.double(), setf_model.val_grads_per_elem.double())#img_query_kernel
                    else:
                        train_val_kernel = kernel(setf_model.grads_per_elem, setf_model.val_grads_per_elem)#img_query_kernel
                    numQuery = train_val_kernel.shape[1]
                    if(len(private_set)!=0): 
                        setf_model_private.compute_gradients(valid=False, batch=False, perClass=False)
                        print("private gradients shape: ", setf_model_private.grads_per_elem.shape)
                        if(doublePrecision):
                            train_private_kernel = kernel(setf_model.grads_per_elem.double(), setf_model_private.grads_per_elem.double()) #img_private_kernel
                        else:
                            train_private_kernel = kernel(setf_model.grads_per_elem, setf_model_private.grads_per_elem) #img_private_kernel
                        numPrivate = train_private_kernel.shape[1]
                    else:
                        train_private_kernel = []
                        numPrivate = 0
                else: # For other submodular functions needing only image kernel
                    setf_model.compute_gradients(valid=False, batch=False, perClass=False)
                    numQueryPrivate = 0

                kernel_time = time.time()
                if(doublePrecision):
                    train_kernel = kernel(setf_model.grads_per_elem.double(), setf_model.grads_per_elem.double()) #img_img_kernel
                else:
                    train_kernel = kernel(setf_model.grads_per_elem, setf_model.grads_per_elem) #img_img_kernel
                if(sf=="logdetmi" or sf=="logdetcg" or sf=="logdetmic"):
                    if(sf=="logdetcg"):
                        if(len(private_set)!=0):
                            val_kernel = kernel(setf_model.val_grads_per_elem, setf_model.val_grads_per_elem)#private_private_kernel
                        else:
                            val_kernel = []
                    if(sf=="logdetmi"):
                        val_kernel = kernel(setf_model.val_grads_per_elem, setf_model.val_grads_per_elem)#query_query_kernel
                    if(sf=="logdetmic"):
                        val_kernel = kernel(setf_model.val_grads_per_elem, setf_model.val_grads_per_elem)#query_query_kernel
                        pp_kernel = kernel(setf_model_private.grads_per_elem, setf_model_private.grads_per_elem)#query_query_kernel
                save_kernel_hdf5(train_kernel, train_val_kernel, val_kernel, train_private_kernel, pp_kernel)
#                 else:
#                     save_kernel_hdf5(train_kernel, train_val_kernel)
                print("kernel compute time: ", time.time()-kernel_time)
                #call the c++ exec to read kernel and compute subset of selected minibatches
                if(sf.endswith("mic")):
                    subset = getCMI_ss(datkbuildPath, exePath, os.getcwd(), budget, str(numQuery), str(numPrivate), sf)
                else:
                    subset = getSMI_ss(datkbuildPath, exePath, os.getcwd(), budget, str(numQueryPrivate), sf)
                print("True targets of subset: ", torch.Tensor(true_lake_set.targets.float())[subset])
                print("Hypothesized targets of subset: ", torch.Tensor(lake_set.targets.float())[subset])
                model.load_state_dict(cached_state_dict)
                if(sf.endswith("cg")): #for first selection
                    if(len(private_set)==0):
                        private_set = custom_subset(lake_set, subset, torch.Tensor(lake_set.targets.float())[subset])
                    else:
                        private_set = getPrivateSet(lake_set, subset, private_set)
                    print("size of private set: ", len(private_set))

    #           temp = np.array(list(trainloader.batch_sampler))[subset] #if per batch
            ###AL###
            elif(strategy=="AL"):
                if(sf=="glister-tss" or sf=="gradmatch-tss" and (feature=="classimb" or feature=="longtail")):
                    miscls_X_val, miscls_y_val = getMisclsSetNumpy(X_val, y_val, val_class_err_idxs, sel_cls_idx)
                    if(sf=="glister-tss"): strategy_sel = GLISTER(X_tr, y_tr, X_unlabeled, model, handler, num_cls, strategy_args, valid=True, X_val=miscls_X_val, Y_val=miscls_y_val, typeOf='rand', lam=0.1)
                    if(sf=="gradmatch-tss"): strategy_sel = GradMatchActive(X_tr, y_tr, X_unlabeled, model, F.cross_entropy, handler, num_cls, strategy_args["lr"], "PerBatch", False, strategy_args, valid=True, X_val=miscls_X_val, Y_val=miscls_y_val)
                    print("reinit AL with targeted miscls samples")
                strategy_sel.update_model(model)
                if(sf=="badge" or sf=="glister" or sf=="glister-tss" or sf=="coreset" or sf=="margin"):
                    subset = strategy_sel.select(budget)
                if(sf=="us" or sf=="leastconf"):
                    subset = list(strategy_sel.select(budget).cpu().numpy())
                if(sf=="gradmatch-tss"):
                    subset = strategy_sel.select(budget, False) #Fixed weight gradmatch
                print(len(subset), " samples selected")
                
            elif(strategy=="random"):
                subset = np.random.choice(np.array(list(range(len(lake_set)))), size=budget, replace=False)
            if(i>1 and sf.endswith("cg")):
                per_ep, overall = check_overlap(prev_idx, prev_idx_hist, list(idx_tracker[subset]))
                per_ep_overlap.append(per_ep)
                overall_overlap.append(overall)
            lake_subset_idxs = subset #indices wrt to lake that need to be removed from the lake
            if(feature=="ood"): #remove ood points from the subset
                subset = remove_ood_points(true_lake_set, subset, sel_cls_idx)
            
            if(strategy=="AL"):#augment train and remove from lake for AL startegies
                X_tr = np.concatenate((X_tr, X_unlabeled[subset]), axis=0)
                X_unlabeled = np.delete(X_unlabeled, lake_subset_idxs, axis = 0)
                y_tr = np.concatenate((y_tr, y_unlabeled[subset]), axis = 0)
                y_unlabeled = np.delete(y_unlabeled, lake_subset_idxs, axis = 0)
                strategy_sel.update_data(X_tr, y_tr, X_unlabeled)
                print("selEpoch: %d, Selection Ended at:" % (i), str(datetime.datetime.now()))
            perClsSel = getPerClassSel(true_lake_set, lake_subset_idxs, num_cls)
            res_dict['sel_per_cls'].append(perClsSel)
            prev_idx = list(idx_tracker[lake_subset_idxs])
            prev_idx_hist += list(idx_tracker[lake_subset_idxs])
            sel_hist.append(list(idx_tracker[lake_subset_idxs]))
            idx_tracker = np.delete(idx_tracker, lake_subset_idxs, axis=0)
            #augment the train_set with selected indices from the lake
            if(feature=="classimb" or feature=="longtail"):
                train_set, lake_set, true_lake_set = aug_train_subset(train_set, lake_set, true_lake_set, subset, lake_subset_idxs, budget, True) #aug train with random if budget is not filled
            elif(feature=="ood"):
                train_set, lake_set, true_lake_set, new_private_set, add_val_set = aug_train_subset(train_set, lake_set, true_lake_set, subset, lake_subset_idxs, budget)
                train_set = torch.utils.data.ConcatDataset([train_set, new_private_set]) #Add the OOD samples with a common OOD class
                val_set = torch.utils.data.ConcatDataset([val_set, add_val_set])
                if(len(private_set)!=0):
                    private_set = torch.utils.data.ConcatDataset([private_set, new_private_set])
                else:
                    private_set = new_private_set
            else:
                train_set, lake_set, true_lake_set = aug_train_subset(train_set, lake_set, true_lake_set, subset, lake_subset_idxs, budget)
            print("After augmentation, size of train_set: ", len(train_set), " lake set: ", len(lake_set))
#           Reinit train and lake loaders with new splits and reinit the model
            trainloader = torch.utils.data.DataLoader(train_set, batch_size=trn_batch_size, shuffle=True, pin_memory=True)
            lakeloader = torch.utils.data.DataLoader(lake_set, batch_size=tst_batch_size, shuffle=False, pin_memory=True)
            valloader = torch.utils.data.DataLoader(val_set, batch_size=val_batch_size, shuffle=False, pin_memory=True)
            assert(len(idx_tracker)==len(lake_set))
#             model =  model.apply(weight_reset).cuda()
            model = create_model(model_name, num_cls, device)
#             optimizer = optimizer_without_scheduler(model, learning_rate)
            optimizer, scheduler = optimizer_with_scheduler(model, 150, learning_rate)
                
        #Start training
        start_time = time.time()
        num_ep=1
        while(full_trn_acc[i]<0.99 and num_ep<150):
            model.train()
            for batch_idx, (inputs, targets) in enumerate(trainloader):
                inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
                # Variables in Pytorch are differentiable.
                inputs, target = Variable(inputs), Variable(inputs)
                # This will zero out the gradients for this batch.
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
            scheduler.step()
          
            full_trn_loss = 0
            full_trn_correct = 0
            full_trn_total = 0
            full_trn_targets = []
            full_trn_outputs = []
            model.eval()
            with torch.no_grad():
                for batch_idx, (inputs, targets) in enumerate(trainloader):
                    inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
                    outputs = model(inputs)
#                     get_roc_auc(targets,outputs,num_cls)
                    loss = criterion(outputs, targets)
                    full_trn_loss += loss.item()
                    _, predicted = outputs.max(1)
                    full_trn_targets += list(targets.cpu().numpy())
                    full_trn_outputs += list(outputs.cpu().numpy().tolist())
                    full_trn_total += targets.size(0)
                    full_trn_correct += predicted.eq(targets).sum().item()
                full_trn_acc[i] = full_trn_correct / full_trn_total
                print("Selection Epoch ", i, " Training epoch [" , num_ep, "]" , " Training Acc: ", full_trn_acc[i], end="\r")
                num_ep+=1
            timing[i] = time.time() - start_time
        print("ROC Curve for train set: \n")
        trn_fpr,trn_tpr,trn_roc_auc = get_roc_auc(full_trn_targets,full_trn_outputs,num_cls)
        trn_precision, trn_recall, trn_aupr = get_pr_auc(full_trn_targets,full_trn_outputs,num_cls)
        res_dict["trn_fpr"].append(trn_fpr)
        res_dict["trn_tpr"].append(trn_tpr)
        res_dict["trn_roc_auc"].append(trn_roc_auc)
        res_dict["trn_precision"].append(trn_precision)
        res_dict["trn_recall"].append(trn_recall)
        res_dict["trn_aupr"].append(trn_aupr)
        with torch.no_grad():
            final_val_targets = []
            final_val_outputs = []
            final_val_predictions = []
            final_val_classifications = []
            for batch_idx, (inputs, targets) in enumerate(valloader):
                # print(batch_idx)
                inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
                outputs = model(inputs)
#                 get_roc_auc(targets,outputs,num_cls)
#                 print('Target: ',targets,'\n Outputs:', outputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += targets.size(0)
                val_correct += predicted.eq(targets).sum().item()
                final_val_targets += list(targets.cpu().numpy())
                final_val_outputs += list(outputs.cpu().numpy().tolist())
    #                 if(i == (num_epochs-1)):
                final_val_predictions += list(predicted.cpu().numpy())
                final_val_classifications += list(predicted.eq(targets).cpu().numpy())
                # sys.exit()

#             if((val_correct/val_total) > best_val_acc):
            final_tst_targets = []
            final_tst_outputs = []
            final_tst_predictions = []
            final_tst_classifications = []
            for batch_idx, (inputs, targets) in enumerate(tstloader):
                inputs, targets = inputs.to(device), targets.to(device, non_blocking=True)
                outputs = model(inputs)
#                 get_roc_auc(targets,outputs,num_cls)
#                 print('Target: ',targets,'\n Outputs:', outputs)
                loss = criterion(outputs, targets)
                tst_loss += loss.item()
                if(feature=="ood"): 
                    _, predicted = outputs[...,:-1].max(1)
                else:
                    _, predicted = outputs.max(1)
                tst_total += targets.size(0)
                tst_correct += predicted.eq(targets).sum().item()
#                 if((val_correct/val_total) > best_val_acc):
    #                 if(i == (num_epochs-1)):
                final_tst_targets += list(targets.cpu().numpy())
                final_tst_outputs += list(outputs.cpu().numpy().tolist())
                final_tst_predictions += list(predicted.cpu().numpy())
                final_tst_classifications += list(predicted.eq(targets).cpu().numpy())
#             if((val_correct/val_total) > best_val_acc):
#                 best_val_acc = (val_correct/val_total)
            print("ROC Curve for validation set: \n")
            val_fpr, val_tpr, val_roc_auc = get_roc_auc(final_val_targets,final_val_outputs,num_cls)
            val_precision, val_recall, val_aupr = get_pr_auc(final_val_targets,final_val_outputs,num_cls)
            res_dict["val_fpr"].append(val_fpr)
            res_dict["val_tpr"].append(val_tpr)
            res_dict["val_roc_auc"].append(val_roc_auc)
            res_dict["val_precision"].append(val_precision)
            res_dict["val_recall"].append(val_recall)
            res_dict["val_aupr"].append(val_aupr)
            print("ROC Curve for test set: \n")
            tst_fpr, tst_tpr, tst_roc_auc = get_roc_auc(final_tst_targets,final_tst_outputs,num_cls)
            tst_precision, tst_recall, tst_aupr = get_pr_auc(final_tst_targets,final_tst_outputs,num_cls)
            macro_avg_roc = get_macro_roc_auc(final_tst_targets,final_tst_outputs)
            res_dict["tst_fpr"].append(tst_fpr)
            res_dict["tst_tpr"].append(tst_tpr)
            res_dict["tst_roc_auc"].append(tst_roc_auc)
            res_dict["tst_precision"].append(tst_precision)
            res_dict["tst_recall"].append(tst_recall)
            res_dict["tst_aupr"].append(tst_aupr)
            res_dict["macro_avg_roc"].append(macro_avg_roc)
            val_acc[i] = val_correct / val_total
            tst_acc[i] = tst_correct / tst_total
            val_losses[i] = val_loss
            fulltrn_losses[i] = full_trn_loss
            tst_losses[i] = tst_loss
            full_val_acc = list(np.array(val_acc))
            full_timing = list(np.array(timing))
            res_dict["test_acc"].append(tst_acc[i])
            
            print('Epoch:', i + 1, 'FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time:', full_trn_loss, full_trn_acc[i], val_loss, val_acc[i], tst_loss, tst_acc[i], timing[i])
        if(i==0): 
            print("saving initial model") 
            torch.save(model.state_dict(), initModelPath) #save initial train model if not present
    if(computeErrorLog):
        tst_err_log, val_err_log, val_class_err_idxs = find_err_per_class(test_set, val_set, final_val_classifications, final_val_predictions, final_tst_classifications, final_tst_predictions, all_logs_dir, sf+"_"+str(bud))
        csvlog.append(tst_err_log)
        val_csvlog.append(val_err_log)
        print(csvlog)
        res_dict["all_class_acc"] = csvlog
        res_dict["all_val_class_acc"] = val_csvlog
#         with open(os.path.join(all_logs_dir, exp_name+".csv"), "w") as f:
#             writer = csv.writer(f)
#             writer.writerows(csvlog)
    #save results dir with test acc and per class selections
    with open(os.path.join(all_logs_dir, exp_name+".json"), 'w') as fp:
        json.dump(res_dict, fp)
    return tst_acc, csvlog

# FL2MI

In [20]:
fl2mi_tst, fl2mi_csvlog = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, 1, device, computeClassErrorLog, "SIM",'fl2mi')

SIM fl2mi
Files already downloaded and verified
Files already downloaded and verified
CIFAR-10 Custom dataset stats: Train size:  3069 Val size:  100 Lake size:  16256
selected classes are:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Saving results to:  active_learning_results/cifar10/longtail/fl2mi/100/1
cifar10_longtail_SIM_10_fl2mi_budget:100_epochs:10_linear:True_runs1
AL epoch:  0
initial training epoch
ROC Curve for train set: ing epoch [ 67 ]  Training Acc:  0.9902248289345064

ROC Curve for validation set: 

ROC Curve for test set: 

Epoch: 1 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 11.662004909245297 0.9902248289345064 33.63485687226057 0.43 3374.031535565853 0.4516 478.19543957710266
saving initial model
AL epoch:  1
val, test error% for class  0  :  100.0 97.8
val, test error% for class  1  :  100.0 98.7
val, test error% for class  2  :  100.0 87.4
val, test error% for class  3  :  100.0 86.5
val, test error% for class  4  :  70.0 67.6
val, test error% for class  5  :  80.0 

TypeError: __init__() takes 6 positional arguments but 8 were given

# FL1MI

In [ ]:
fl1mi_tst, fl1mi_csvlog = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, 1, device, computeClassErrorLog, "SIM",'fl1mi')

# BADGE

In [17]:
# train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, 1, device, False, "SIM",'gccg')
badge_tst, badge_csvlog = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, run, device, computeClassErrorLog, "AL","badge")

AL badge
Files already downloaded and verified
Files already downloaded and verified
CIFAR-10 Custom dataset stats: Train size:  3069 Val size:  100 Lake size:  16256
selected classes are:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Saving results to:  active_learning_results/cifar10/longtail/badge/250/1
cifar10_longtail_AL_10_badge_budget:250_epochs:10_linear:True_runs1
AL epoch:  0
initial training epoch
Init model loaded from disk, skipping init training:  weights/cifar10_ResNet18_0.01_{0: 3, 1: 6, 2: 12, 3: 24, 4: 48, 5: 96, 6: 192, 7: 384, 8: 768, 9: 1536}
AL epoch:  1
val, test error% for class  0  :  100.0 98.8
val, test error% for class  1  :  100.0 99.7
val, test error% for class  2  :  100.0 95.1
val, test error% for class  3  :  100.0 94.3
val, test error% for class  4  :  90.0 77.1
val, test error% for class  5  :  80.0 72.0
val, test error% for class  6  :  90.0 52.7
val, test error% for class  7  :  90.0 21.0
val, test error% for class  8  :  30.0 9.6
val, test error% for class  9  :

250  samples selected
selEpoch: 9, Selection Ended at: 2021-06-19 14:23:05.710023
250 14006 14256
After augmentation, size of train_set:  5319  lake set:  14006
ROC Curve for train set: ing epoch [ 78 ]  Training Acc:  0.9952998683963151

ROC Curve for validation set: 

ROC Curve for test set: 

Epoch: 10 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 11.880913501539908 0.9952998683963151 13.889364654547535 0.7 1575.1706732516177 0.686 970.0724008083344
val, test error% for class  0  :  70.0 59.4
val, test error% for class  1  :  80.0 63.8
val, test error% for class  2  :  50.0 56.1
val, test error% for class  3  :  40.0 58.0
val, test error% for class  4  :  10.0 27.7
val, test error% for class  5  :  10.0 27.8
val, test error% for class  6  :  10.0 8.3
val, test error% for class  7  :  20.0 6.4
val, test error% for class  8  :  0.0 3.3
val, test error% for class  9  :  10.0 3.2
[[98.8, 99.7, 95.1, 94.3, 77.1, 72.0, 52.7, 21.0, 9.6, 3.5, 62.38000000000001], [89.6, 96.2, 82.8, 88

# US

In [18]:
us_tst, us_csvlog = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, run, device, computeClassErrorLog, "AL","us")

AL us
Files already downloaded and verified
Files already downloaded and verified
CIFAR-10 Custom dataset stats: Train size:  3069 Val size:  100 Lake size:  16256
selected classes are:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Saving results to:  active_learning_results/cifar10/longtail/us/250/1
cifar10_longtail_AL_10_us_budget:250_epochs:10_linear:True_runs1
AL epoch:  0
initial training epoch
Init model loaded from disk, skipping init training:  weights/cifar10_ResNet18_0.01_{0: 3, 1: 6, 2: 12, 3: 24, 4: 48, 5: 96, 6: 192, 7: 384, 8: 768, 9: 1536}
AL epoch:  1
val, test error% for class  0  :  100.0 98.8
val, test error% for class  1  :  100.0 99.7
val, test error% for class  2  :  100.0 95.1
val, test error% for class  3  :  100.0 94.3
val, test error% for class  4  :  100.0 77.1
val, test error% for class  5  :  80.0 72.0
val, test error% for class  6  :  80.0 52.7
val, test error% for class  7  :  80.0 21.0
val, test error% for class  8  :  30.0 9.6
val, test error% for class  9  :  10.0 3

250  samples selected
selEpoch: 9, Selection Ended at: 2021-06-19 16:20:55.262226
250 14006 14256
After augmentation, size of train_set:  5319  lake set:  14006
ROC Curve for train set: ing epoch [ 84 ]  Training Acc:  0.9913517578492198

ROC Curve for validation set: 

ROC Curve for test set: 

Epoch: 10 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 17.267263524088776 0.9913517578492198 15.332551185972989 0.66 1649.1507547274232 0.6867 1042.8926486968994
val, test error% for class  0  :  70.0 54.4
val, test error% for class  1  :  90.0 74.3
val, test error% for class  2  :  60.0 48.9
val, test error% for class  3  :  60.0 54.8
val, test error% for class  4  :  10.0 26.6
val, test error% for class  5  :  10.0 26.0
val, test error% for class  6  :  0.0 10.8
val, test error% for class  7  :  40.0 6.4
val, test error% for class  8  :  0.0 8.6
val, test error% for class  9  :  0.0 2.5
[[98.8, 99.7, 95.1, 94.3, 77.1, 72.0, 52.7, 21.0, 9.6, 3.5, 62.38000000000001], [92.3, 97.6, 91.2, 

# GLISTER

In [22]:
us_tst, us_csvlog = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, run, device, computeClassErrorLog, "AL","glister-tss")

AL glister-tss
Files already downloaded and verified
Files already downloaded and verified
CIFAR-10 Custom dataset stats: Train size:  3069 Val size:  100 Lake size:  16256
selected classes are:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Saving results to:  active_learning_results/cifar10/longtail/glister-tss/250/1
cifar10_longtail_AL_10_glister-tss_budget:250_epochs:10_linear:True_runs1
AL epoch:  0
initial training epoch
Init model loaded from disk, skipping init training:  weights/cifar10_ResNet18_0.01_{0: 3, 1: 6, 2: 12, 3: 24, 4: 48, 5: 96, 6: 192, 7: 384, 8: 768, 9: 1536}
AL epoch:  1
val, test error% for class  0  :  100.0 98.8
val, test error% for class  1  :  100.0 99.7
val, test error% for class  2  :  100.0 95.1
val, test error% for class  3  :  90.0 94.3
val, test error% for class  4  :  100.0 77.1
val, test error% for class  5  :  80.0 72.0
val, test error% for class  6  :  90.0 52.7
val, test error% for class  7  :  80.0 21.0
val, test error% for class  8  :  60.0 9.6
val, test erro

/scratch4/snk170001/code/distil/distil/active_learning_strategies/strategy.py:151: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_trn = torch.tensor(Y[idxs])
/scratch4/snk170001/code/distil/distil/active_learning_strategies/glister.py:206: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_Val = torch.tensor(self.Y_Val,device=self.device)


250  samples selected
selEpoch: 1, Selection Ended at: 2021-06-20 01:01:56.701314
250 16006 16256
After augmentation, size of train_set:  3319  lake set:  16006
ROC Curve for train set: ing epoch [ 71 ]  Training Acc:  0.9957818620066285

ROC Curve for validation set: 

ROC Curve for test set: 

Epoch: 2 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 6.379328084098233 0.9957818620066285 24.021292354911566 0.56 2890.6853822860867 0.5256 561.0333876609802
AL epoch:  2
val, test error% for class  0  :  60.0 73.0
val, test error% for class  1  :  100.0 93.4
val, test error% for class  2  :  70.0 77.5
val, test error% for class  3  :  80.0 79.5
val, test error% for class  4  :  70.0 72.0
val, test error% for class  5  :  40.0 40.3
val, test error% for class  6  :  0.0 13.1
val, test error% for class  7  :  10.0 17.1
val, test error% for class  8  :  10.0 6.0
val, test error% for class  9  :  0.0 2.5
total misclassified ex from imb classes:  44
reinit AL with targeted miscls samples


/scratch4/snk170001/code/distil/distil/active_learning_strategies/glister.py:206: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_Val = torch.tensor(self.Y_Val,device=self.device)


250  samples selected
selEpoch: 2, Selection Ended at: 2021-06-20 01:11:44.590799
250 15756 16006
After augmentation, size of train_set:  3569  lake set:  15756
ROC Curve for train set: ing epoch [ 73 ]  Training Acc:  0.9921546651723172

ROC Curve for validation set: 

ROC Curve for test set: 

Epoch: 3 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 10.7875901843654 0.9921546651723172 25.038797814399004 0.57 2639.6917184628546 0.5624 619.014411687851
AL epoch:  3
val, test error% for class  0  :  70.0 61.0
val, test error% for class  1  :  100.0 86.9
val, test error% for class  2  :  70.0 76.0
val, test error% for class  3  :  80.0 82.1
val, test error% for class  4  :  50.0 55.0
val, test error% for class  5  :  30.0 33.5
val, test error% for class  6  :  0.0 18.1
val, test error% for class  7  :  0.0 8.2
val, test error% for class  8  :  10.0 13.0
val, test error% for class  9  :  20.0 3.8
total misclassified ex from imb classes:  43
reinit AL with targeted miscls samples


...

250  samples selected
selEpoch: 7, Selection Ended at: 2021-06-20 02:13:59.782873
250 14506 14756
After augmentation, size of train_set:  4819  lake set:  14506
ROC Curve for train set: ing epoch [ 77 ]  Training Acc:  0.9904544511309439

ROC Curve for validation set: 

ROC Curve for test set: 

Epoch: 8 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 18.03445039914368 0.99045445113094 14.638382891193032 0.69 1398.0263657253236 0.7009 893.2568829059601
AL epoch:  8
val, test error% for class  0  :  60.0 58.2
val, test error% for class  1  :  70.0 46.2
val, test error% for class  2  :  60.0 54.6
val, test error% for class  3  :  70.0 57.0
val, test error% for class  4  :  10.0 33.2
val, test error% for class  5  :  20.0 24.2
val, test error% for class  6  :  10.0 8.8
val, test error% for class  7  :  0.0 10.4
val, test error% for class  8  :  0.0 2.9
val, test error% for class  9  :  10.0 3.6
total misclassified ex from imb classes:  31
reinit AL with targeted miscls samples


/scratch4/snk170001/code/distil/distil/active_learning_strategies/glister.py:206: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_Val = torch.tensor(self.Y_Val,device=self.device)


250  samples selected
selEpoch: 8, Selection Ended at: 2021-06-20 02:29:20.216569
250 14256 14506
After augmentation, size of train_set:  5069  lake set:  14256
ROC Curve for train set: ing epoch [ 78 ]  Training Acc:  0.9901361215229828

ROC Curve for validation set: 

ROC Curve for test set: 

Epoch: 9 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 16.65276038005686 0.9901361215229828 16.70490389841143 0.65 1503.1723329448141 0.6925 935.3542416095734
AL epoch:  9
val, test error% for class  0  :  80.0 61.7
val, test error% for class  1  :  70.0 51.6
val, test error% for class  2  :  70.0 49.8
val, test error% for class  3  :  60.0 47.8
val, test error% for class  4  :  30.0 37.2
val, test error% for class  5  :  20.0 35.8
val, test error% for class  6  :  0.0 8.4
val, test error% for class  7  :  10.0 8.6
val, test error% for class  8  :  0.0 3.5
val, test error% for class  9  :  10.0 3.1
total misclassified ex from imb classes:  35
reinit AL with targeted miscls samples


/scratch4/snk170001/code/distil/distil/active_learning_strategies/glister.py:206: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_Val = torch.tensor(self.Y_Val,device=self.device)


250  samples selected
selEpoch: 9, Selection Ended at: 2021-06-20 02:45:22.481541
250 14006 14256
After augmentation, size of train_set:  5319  lake set:  14006
ROC Curve for train set: ing epoch [ 76 ]  Training Acc:  0.9904117315284828

ROC Curve for validation set: 

ROC Curve for test set: 

Epoch: 10 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 18.839684868115 0.9904117315284828 13.314054071903229 0.66 1290.6160087757744 0.7132 959.8735342025757
val, test error% for class  0  :  70.0 65.9
val, test error% for class  1  :  70.0 34.9
val, test error% for class  2  :  50.0 59.4
val, test error% for class  3  :  50.0 45.3
val, test error% for class  4  :  30.0 26.7
val, test error% for class  5  :  30.0 31.8
val, test error% for class  6  :  0.0 8.0
val, test error% for class  7  :  20.0 7.4
val, test error% for class  8  :  10.0 2.4
val, test error% for class  9  :  10.0 5.0
[[98.8, 99.7, 95.1, 94.3, 77.1, 72.0, 52.7, 21.0, 9.6, 3.5, 62.38000000000001], [73.0, 93.4, 77.5, 79.

# GradMatch-Active

In [ ]:
gradmatch_tst, gradmatch_csvlog = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, 1, device, computeClassErrorLog, "AL","gradmatch-tss")

# Coreset

In [19]:
coreset_tst, coreset_csvlog = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, run, device, computeClassErrorLog, "AL","coreset")

AL coreset
Files already downloaded and verified
Files already downloaded and verified
CIFAR-10 Custom dataset stats: Train size:  3069 Val size:  100 Lake size:  16256
selected classes are:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Saving results to:  active_learning_results/cifar10/longtail/coreset/250/1
cifar10_longtail_AL_10_coreset_budget:250_epochs:10_linear:True_runs1
AL epoch:  0
initial training epoch
Init model loaded from disk, skipping init training:  weights/cifar10_ResNet18_0.01_{0: 3, 1: 6, 2: 12, 3: 24, 4: 48, 5: 96, 6: 192, 7: 384, 8: 768, 9: 1536}
AL epoch:  1
val, test error% for class  0  :  100.0 98.8
val, test error% for class  1  :  100.0 99.7
val, test error% for class  2  :  100.0 95.1
val, test error% for class  3  :  90.0 94.3
val, test error% for class  4  :  90.0 77.1
val, test error% for class  5  :  80.0 72.0
val, test error% for class  6  :  90.0 52.7
val, test error% for class  7  :  80.0 21.0
val, test error% for class  8  :  50.0 9.6
val, test error% for class 

250  samples selected
selEpoch: 9, Selection Ended at: 2021-06-19 18:09:07.440750
250 14006 14256
After augmentation, size of train_set:  5319  lake set:  14006
ROC Curve for train set: ing epoch [ 73 ]  Training Acc:  0.9904117315284828

ROC Curve for validation set: 

ROC Curve for test set: 

Epoch: 10 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 17.024865324696293 0.9904117315284828 12.562836399301887 0.68 1379.333356671501 0.6862 910.4367280006409
val, test error% for class  0  :  80.0 59.5
val, test error% for class  1  :  50.0 43.7
val, test error% for class  2  :  40.0 58.6
val, test error% for class  3  :  80.0 61.7
val, test error% for class  4  :  30.0 37.7
val, test error% for class  5  :  20.0 21.6
val, test error% for class  6  :  10.0 13.3
val, test error% for class  7  :  0.0 9.5
val, test error% for class  8  :  10.0 4.9
val, test error% for class  9  :  0.0 3.3
[[98.8, 99.7, 95.1, 94.3, 77.1, 72.0, 52.7, 21.0, 9.6, 3.5, 62.38000000000001], [98.0, 95.2, 83.5, 9

# Least Confidence

In [20]:
leastconf_tst, leastconf_csvlog = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, run, device, computeClassErrorLog, "AL","leastconf")

AL leastconf
Files already downloaded and verified
Files already downloaded and verified
CIFAR-10 Custom dataset stats: Train size:  3069 Val size:  100 Lake size:  16256
selected classes are:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Saving results to:  active_learning_results/cifar10/longtail/leastconf/250/1
cifar10_longtail_AL_10_leastconf_budget:250_epochs:10_linear:True_runs1
AL epoch:  0
initial training epoch
Init model loaded from disk, skipping init training:  weights/cifar10_ResNet18_0.01_{0: 3, 1: 6, 2: 12, 3: 24, 4: 48, 5: 96, 6: 192, 7: 384, 8: 768, 9: 1536}
AL epoch:  1
val, test error% for class  0  :  100.0 98.8
val, test error% for class  1  :  100.0 99.7
val, test error% for class  2  :  90.0 95.1
val, test error% for class  3  :  90.0 94.3
val, test error% for class  4  :  90.0 77.1
val, test error% for class  5  :  90.0 72.0
val, test error% for class  6  :  80.0 52.7
val, test error% for class  7  :  80.0 21.0
val, test error% for class  8  :  30.0 9.6
val, test error% for c

250  samples selected
selEpoch: 9, Selection Ended at: 2021-06-19 20:06:33.594653
250 14006 14256
After augmentation, size of train_set:  5319  lake set:  14006
ROC Curve for train set: ing epoch [ 83 ]  Training Acc:  0.9902237262643354

ROC Curve for validation set: 

ROC Curve for test set: 

Epoch: 10 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 19.334926577532315 0.9902237262643354 18.395845658611506 0.67 1787.3371587841684 0.6837 1017.254768371582
val, test error% for class  0  :  60.0 53.0
val, test error% for class  1  :  80.0 77.8
val, test error% for class  2  :  70.0 68.3
val, test error% for class  3  :  50.0 44.9
val, test error% for class  4  :  20.0 20.9
val, test error% for class  5  :  10.0 27.9
val, test error% for class  6  :  0.0 7.9
val, test error% for class  7  :  20.0 7.6
val, test error% for class  8  :  0.0 4.4
val, test error% for class  9  :  20.0 3.6
[[98.8, 99.7, 95.1, 94.3, 77.1, 72.0, 52.7, 21.0, 9.6, 3.5, 62.38000000000001], [93.1, 98.4, 75.7, 8

# Margin Sampling

In [21]:
margin_tst, margin_csvlog = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, run, device, computeClassErrorLog, "AL","margin")

AL margin
Files already downloaded and verified
Files already downloaded and verified
CIFAR-10 Custom dataset stats: Train size:  3069 Val size:  100 Lake size:  16256
selected classes are:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Saving results to:  active_learning_results/cifar10/longtail/margin/250/1
cifar10_longtail_AL_10_margin_budget:250_epochs:10_linear:True_runs1
AL epoch:  0
initial training epoch
Init model loaded from disk, skipping init training:  weights/cifar10_ResNet18_0.01_{0: 3, 1: 6, 2: 12, 3: 24, 4: 48, 5: 96, 6: 192, 7: 384, 8: 768, 9: 1536}
AL epoch:  1
val, test error% for class  0  :  100.0 98.8
val, test error% for class  1  :  100.0 99.7
val, test error% for class  2  :  90.0 95.1
val, test error% for class  3  :  100.0 94.3
val, test error% for class  4  :  90.0 77.1
val, test error% for class  5  :  90.0 72.0
val, test error% for class  6  :  90.0 52.7
val, test error% for class  7  :  90.0 21.0
val, test error% for class  8  :  40.0 9.6
val, test error% for class  9 

250  samples selected
selEpoch: 9, Selection Ended at: 2021-06-19 22:02:31.548147
250 14006 14256
After augmentation, size of train_set:  5319  lake set:  14006
ROC Curve for train set: ing epoch [ 83 ]  Training Acc:  0.9926677946982515

ROC Curve for validation set: 

ROC Curve for test set: 

Epoch: 10 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 15.4223115209752 0.9926677946982515 17.397684478200972 0.64 1666.3359357242007 0.6781 1029.2016246318817
val, test error% for class  0  :  70.0 60.1
val, test error% for class  1  :  70.0 57.3
val, test error% for class  2  :  80.0 72.4
val, test error% for class  3  :  70.0 55.5
val, test error% for class  4  :  30.0 33.5
val, test error% for class  5  :  30.0 22.7
val, test error% for class  6  :  0.0 6.4
val, test error% for class  7  :  10.0 7.2
val, test error% for class  8  :  0.0 5.3
val, test error% for class  9  :  0.0 1.5
[[98.8, 99.7, 95.1, 94.3, 77.1, 72.0, 52.7, 21.0, 9.6, 3.5, 62.38000000000001], [95.5, 97.8, 86.3, 85.

# GCMI+DIV

In [ ]:
gcmidiv_tst, gcmidiv_csvlog = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, run, device, computeClassErrorLog, "SIM",'div-gcmi')

# GCMI

In [ ]:
gcmi_tst, gcmi_csvlog = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, run, device, computeClassErrorLog, "SIM",'gcmi')

# LOGDETMI

In [33]:
logdetmi_tst, logdetmi_csvlog = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, run, device, computeClassErrorLog, "SIM",'logdetmi')

SIM logdetmi
Breast-density Custom dataset stats: Train size:  241 Val size:  40 Lake size:  2139
selected classes are:  [0, 3]
Saving results to:  SMI_active_learning_results_woVal/breast_density/classimb/logdetmi/50/1
breast_density_classimb_SIM_2_logdetmi_budget:50_epochs:20_linear:True_runs1
AL epoch:  0
initial training epoch
Init model loaded from disk, skipping init training:  weights/breast_density_MobileNetV2_0.01_{0: 35, 3: 40}_{1: 93, 2: 73}
AL epoch:  1
val, test error% for class  0  :  90.0 89.23
val, test error% for class  1  :  60.0 46.15
val, test error% for class  2  :  80.0 72.64
val, test error% for class  3  :  80.0 86.73
total misclassified ex from imb classes:  31
Per Element Training Gradient Computation is Completed
Per Element Validation Gradient Computation is Completed
train minibatch gradients shape  torch.Size([2139, 5124])
val minibatch gradients shape  torch.Size([31, 5124])
kernel compute time:  0.8748178482055664
Executing SIM command:  /home/snk170001/

True targets of subset:  tensor([1., 0., 1., 2., 0., 2., 2., 1., 2., 1., 2., 1., 2., 2., 1., 1., 2., 3.,
        2., 2., 3., 1., 1., 1., 2., 2., 2., 1., 1., 2., 1., 2., 0., 1., 1., 1.,
        3., 3., 0., 0., 3., 1., 1., 3., 2., 3., 3., 2., 2., 1.])
Hypothesized targets of subset:  tensor([1., 2., 1., 1., 0., 1., 2., 0., 1., 3., 2., 2., 2., 2., 0., 1., 1., 3.,
        3., 2., 3., 2., 0., 1., 1., 1., 3., 0., 0., 2., 1., 2., 0., 0., 2., 1.,
        3., 2., 0., 0., 2., 0., 0., 2., 3., 3., 2., 3., 1., 2.])
50 1889 1939
After augmentation, size of train_set:  491  lake set:  1889
Epoch: 6 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 43.30398443341255 0.6272912423625254 3.7448381781578064 0.5 65.81571018695831 0.5289514866979655 3235.0083799362183
AL epoch:  6
val, test error% for class  0  :  50.0 36.92
val, test error% for class  1  :  60.0 51.92
val, test error% for class  2  :  60.0 47.26
val, test error% for class  3  :  30.0 41.59
total misclassified ex from imb classes:  20
Pe

kernel compute time:  0.6493377685546875
Executing SIM command:  /home/snk170001/bioml/dss/notebooks/datk/build/cifarSubsetSelector_ng -mode query -naiveOrRandom naive -logDetLambda 1 -magnificationLambda 1 -numSummaries 1 -budget 50 -queryPrivacyOptimizer logdetmi -numQueries  17  -dontComputeKernel true -imageKernelFile /home/snk170001/bioml/dss/notebooks/smi_lake_kernel_1.hdf5 -queryKernelFile /home/snk170001/bioml/dss/notebooks/smi_lake_target_kernel_1.hdf5 -queryqueryKernelFile /home/snk170001/bioml/dss/notebooks/smi_target_kernel_1.hdf5
True targets of subset:  tensor([2., 2., 1., 1., 3., 2., 3., 3., 0., 2., 1., 0., 1., 1., 3., 1., 3., 1.,
        3., 1., 1., 0., 0., 3., 1., 2., 3., 3., 1., 1., 3., 1., 3., 1., 3., 0.,
        1., 3., 2., 3., 3., 2., 1., 1., 2., 1., 2., 0., 2., 2.])
Hypothesized targets of subset:  tensor([1., 2., 3., 1., 3., 2., 0., 2., 0., 1., 0., 0., 0., 1., 2., 0., 3., 1.,
        3., 1., 1., 0., 0., 3., 0., 2., 1., 3., 2., 0., 3., 1., 2., 1., 3., 0.,
        

Epoch: 15 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 82.72060441970825 0.6323060573857598 3.2334787845611572 0.625 61.430308520793915 0.568075117370892 6534.424946784973
AL epoch:  15
val, test error% for class  0  :  30.0 29.23
val, test error% for class  1  :  40.0 47.31
val, test error% for class  2  :  50.0 47.76
val, test error% for class  3  :  30.0 33.63
total misclassified ex from imb classes:  15
Per Element Training Gradient Computation is Completed
Per Element Validation Gradient Computation is Completed
train minibatch gradients shape  torch.Size([1439, 5124])
val minibatch gradients shape  torch.Size([15, 5124])
kernel compute time:  0.5626442432403564
Executing SIM command:  /home/snk170001/bioml/dss/notebooks/datk/build/cifarSubsetSelector_ng -mode query -naiveOrRandom naive -logDetLambda 1 -magnificationLambda 1 -numSummaries 1 -budget 50 -queryPrivacyOptimizer logdetmi -numQueries  15  -dontComputeKernel true -imageKernelFile /home/snk170001/bioml/dss/noteboo

True targets of subset:  tensor([1., 0., 3., 1., 2., 1., 0., 3., 2., 3., 0., 1., 2., 2., 2., 1., 2., 3.,
        3., 1., 2., 2., 1., 1., 0., 1., 1., 2., 2., 0., 1., 3., 1., 2., 1., 1.,
        2., 1., 1., 1., 2., 2., 1., 2., 3., 1., 2., 1., 1., 2.])
Hypothesized targets of subset:  tensor([1., 0., 3., 2., 1., 0., 1., 3., 1., 2., 0., 1., 2., 2., 2., 0., 2., 1.,
        3., 0., 2., 0., 2., 2., 1., 1., 0., 2., 2., 0., 1., 3., 1., 3., 1., 0.,
        1., 0., 1., 0., 2., 2., 1., 2., 3., 0., 1., 1., 0., 2.])
50 1189 1239
After augmentation, size of train_set:  1191  lake set:  1189
Epoch: 20 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 93.03159859776497 0.6809403862300588 2.8329659700393677 0.7 57.633639603853226 0.6087636932707355 8080.754028558731
val, test error% for class  0  :  30.0 33.85
val, test error% for class  1  :  50.0 40.38
val, test error% for class  2  :  30.0 43.78
val, test error% for class  3  :  10.0 30.97
[[89.23, 46.15, 72.64, 86.73, 73.6875], [50.77, 46.15, 44.

# FL

In [ ]:
fl_tst, fl_csvlog = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, run, device, computeClassErrorLog, "SF",'fl')

# GC

In [ ]:
gc_tst, gc_csvlog = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, run, device, computeClassErrorLog, "SF",'gc')

# LOGDET

In [ ]:
logdet_tst, logdet_csvlog = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, run, device, computeClassErrorLog, "SF",'logdet')

# Random

In [36]:
# for i in range(1,6):
random_test_acc = train_model_al(datkbuildPath, exePath, num_epochs, data_name, datadir, feature, model_name, budget, split_cfg, learning_rate, run, device, computeClassErrorLog, "random",'random')

random random
Breast-density Custom dataset stats: Train size:  261 Val size:  40 Lake size:  2139
selected classes are:  [0, 3]
Saving results to:  SMI_active_learning_results_woVal/breast_density/classimb/random/50/2
breast_density_classimb_random_2_random_budget:50_epochs:20_linear:True_runs2
AL epoch:  0
initial training epoch
Init model loaded from disk, skipping init training:  weights/breast_density_MobileNetV2_0.01_{0: 35, 3: 40}_{1: 93, 2: 73}
AL epoch:  1
val, test error% for class  0  :  80.0 89.23
val, test error% for class  1  :  50.0 46.15
val, test error% for class  2  :  70.0 72.64
val, test error% for class  3  :  100.0 86.73
50 2089 2139
After augmentation, size of train_set:  311  lake set:  2089
Epoch: 2 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 31.729574978351593 0.572347266881029 3.69535756111145 0.575 70.8421282172203 0.47104851330203446 2019.6698310375214
AL epoch:  2
val, test error% for class  0  :  40.0 30.77
val, test error% for class  1  :  60.0 

Epoch: 19 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 97.54228103160858 0.632213608957795 2.8004336059093475 0.75 58.62907028198242 0.5821596244131455 7615.39546918869
AL epoch:  19
val, test error% for class  0  :  40.0 46.15
val, test error% for class  1  :  0.0 36.54
val, test error% for class  2  :  60.0 51.24
val, test error% for class  3  :  0.0 34.51
50 1189 1239
After augmentation, size of train_set:  1211  lake set:  1189
Epoch: 20 FullTrn,TrainAcc,ValLoss,ValAcc,TstLoss,TstAcc,Time: 104.68075922131538 0.6077621800165153 3.2187494337558746 0.65 57.81700223684311 0.5727699530516432 7946.918114185333
val, test error% for class  0  :  50.0 47.69
val, test error% for class  1  :  30.0 43.08
val, test error% for class  2  :  40.0 42.29
val, test error% for class  3  :  20.0 39.82
[[89.23, 46.15, 72.64, 86.73, 73.6875], [30.77, 56.92, 65.17, 34.51, 46.8425], [40.0, 40.0, 65.67, 40.71, 46.595000000000006], [13.85, 66.92, 95.02, 99.12, 68.72749999999999], [30.77, 42.69, 71.14